In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model, Input
from tensorflow.keras import layers
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from pandas import concat
from pandas import read_csv
from helper import series_to_supervised, stage_series_to_supervised

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
dataset = pd.read_csv('../../data/TFT_energy.csv', index_col=0)
dataset.fillna(0, inplace=True)
data = dataset
data

,price,price_dayahead,gen_coal,gen_gas,load_actual,gen_lig,gen_oil,gen_oth_renew,pressure_Barcelona,pressure_Bilbao,...,wind_deg_Bilbao,clouds_all_Bilbao,gen_hyd_river,wind_deg_Seville,wind_speed_Barcelona,wind_speed_Valencia,wind_speed_Bilbao,gen_wind,wind_speed_Madrid,gen_hyd_pump
time,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00+00:00,64.92,48.10,4755.0,5196.0,24382.0,328.0,158.0,71.0,1035.0,1035.0,...,229.0,0.0,1009.0,21.0,7.0,1.0,0.0,5890.0,1.0,920.0
2015-01-01 01:00:00+00:00,64.48,47.33,4581.0,4857.0,22734.0,323.0,157.0,73.0,1036.0,1036.0,...,224.0,0.0,973.0,27.0,7.0,0.0,1.0,5461.0,1.0,1164.0
2015-01-01 02:00:00+00:00,59.32,42.27,4131.0,4314.0,21286.0,254.0,160.0,75.0,1036.0,1035.0,...,225.0,0.0,949.0,27.0,7.0,0.0,1.0,5238.0,1.0,1503.0
2015-01-01 03:00:00+00:00,56.04,38.41,3840.0,4130.0,20264.0,187.0,156.0,74.0,1036.0,1035.0,...,221.0,0.0,953.0,27.0,7.0,0.0,1.0,4935.0,1.0,1826.0
2015-01-01 04:00:00+00:00,53.63,35.72,3590.0,4038.0,19905.0,178.0,156.0,74.0,1037.0,1035.0,...,224.0,0.0,952.0,57.0,5.0,2.0,1.0,4618.0,0.0,2109.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 18:00:00+00:00,77.02,68.85,2628.0,7634.0,30653.0,0.0,178.0,95.0,1027.0,1033.0,...,57.0,0.0,1135.0,30.0,1.0,2.0,0.0,3113.0,1.0,1.0
2018-12-31 19:00:00+00:00,76.16,68.40,2566.0,7241.0,29735.0,0.0,174.0,95.0,1027.0,1034.0,...,0.0,0.0,1172.0,30.0,3.0,1.0,1.0,3288.0,1.0,1.0
2018-12-31 20:00:00+00:00,74.30,66.88,2422.0,7025.0,28071.0,0.0,168.0,94.0,1028.0,1034.0,...,140.0,0.0,1148.0,50.0,4.0,3.0,1.0,3503.0,1.0,50.0


In [4]:
print(dataset.columns)

Index(['price', 'price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump'],
      dtype='object')


In [5]:
# specify the number of lag hours
n_hours = 24*3
K = 24

### Target time series (4)
- 'WS_S1',  'TWS_S25A', 'TWS_S25B', 'TWS_S26',    
        
               
### Prior known covariates (9)
- 'WS_S4', 'GATE_S25A', 'GATE_S25B', 'GATE_S25B2', 'PUMP_S25B',  'GATE_S26_1', 'GATE_S26_2', 'PUMP_S26', 'MEAN_RAIN'


### Prior unknown covariates (6)
- 'FLOW_S25A', 'HWS_S25A', 'FLOW_S25B', 'HWS_S25B', 'FLOW_S26', 'HWS_S26'

#### Target time series: water stage (4)

In [6]:
# Pre-processing
stages = dataset[['price']]
print("stages.shape:", stages.shape)


stages_supervised = series_to_supervised(stages, n_hours, K)
print("stages_supervised.shape:", stages_supervised.shape)

stages.shape: (35063, 1)
stages_supervised.shape: (34968, 96)


In [7]:
stages_supervised

,var1(t-72),var1(t-71),var1(t-70),var1(t-69),var1(t-68),var1(t-67),var1(t-66),var1(t-65),var1(t-64),var1(t-63),...,var1(t+14),var1(t+15),var1(t+16),var1(t+17),var1(t+18),var1(t+19),var1(t+20),var1(t+21),var1(t+22),var1(t+23)
time,,,,,,,,,,,,,,,,,,,,,
2015-01-04 00:00:00+00:00,64.92,64.48,59.32,56.04,53.63,51.73,51.43,48.98,54.20,58.94,...,52.07,52.15,61.12,71.98,74.64,75.38,76.49,73.83,69.51,70.77
2015-01-04 01:00:00+00:00,64.48,59.32,56.04,53.63,51.73,51.43,48.98,54.20,58.94,59.86,...,52.15,61.12,71.98,74.64,75.38,76.49,73.83,69.51,70.77,64.89
2015-01-04 02:00:00+00:00,59.32,56.04,53.63,51.73,51.43,48.98,54.20,58.94,59.86,60.12,...,61.12,71.98,74.64,75.38,76.49,73.83,69.51,70.77,64.89,60.91
2015-01-04 03:00:00+00:00,56.04,53.63,51.73,51.43,48.98,54.20,58.94,59.86,60.12,62.05,...,71.98,74.64,75.38,76.49,73.83,69.51,70.77,64.89,60.91,59.68
2015-01-04 04:00:00+00:00,53.63,51.73,51.43,48.98,54.20,58.94,59.86,60.12,62.05,62.06,...,74.64,75.38,76.49,73.83,69.51,70.77,64.89,60.91,59.68,58.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-30 19:00:00+00:00,78.99,77.65,73.28,73.48,70.93,68.61,64.19,62.53,59.78,59.82,...,72.12,71.81,72.42,71.80,71.95,70.85,71.36,75.13,77.61,77.02
2018-12-30 20:00:00+00:00,77.65,73.28,73.48,70.93,68.61,64.19,62.53,59.78,59.82,64.27,...,71.81,72.42,71.80,71.95,70.85,71.36,75.13,77.61,77.02,76.16
2018-12-30 21:00:00+00:00,73.28,73.48,70.93,68.61,64.19,62.53,59.78,59.82,64.27,66.81,...,72.42,71.80,71.95,70.85,71.36,75.13,77.61,77.02,76.16,74.30


#### Prior unknown covariates (6)

In [8]:
# prior_unknown = dataset[['FLOW_S25A', 'HWS_S25A', 'FLOW_S25B', 'HWS_S25B', 'FLOW_S26', 'HWS_S26' ]]
# print("prior_unknown.shape:", prior_unknown.shape)

# prior_unknown_supervised = series_to_supervised(prior_unknown, n_hours, K)
# print("prior_unknown_supervised.shape:", prior_unknown_supervised.shape)

In [9]:
# prior_unknown_supervised

#### Prior known covariates (9)

In [10]:
prior_known = dataset[['price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump']]
print("prior_known.shape:", prior_known.shape)

prior_known_supervised = series_to_supervised(prior_known, n_hours, K)
print("prior_known_supervised.shape:", prior_known_supervised.shape)

prior_known.shape: (35063, 25)
prior_known_supervised.shape: (34968, 2400)


In [11]:
# prior_known_supervised

### N_out, N_in, K

In [12]:
past_cov = dataset[['price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump']]

past_cov_supervised = series_to_supervised(past_cov, n_hours, K)
print("past_cov_supervised.shape:", past_cov_supervised.shape)

past_cov_supervised.shape: (34968, 2400)


In [13]:
# past_cov_supervised  # 6 + 9 = 15 features

In [14]:
past_ws_supervised = stages_supervised.iloc[:, :n_hours*stages.shape[1]]
past_ws_supervised.reset_index(drop=True, inplace=True)
past_ws_supervised.columns = ['past_ws_supervised_' + i for i in list(past_ws_supervised.columns)]

In [15]:
# past_ws_supervised  # 4

In [16]:
columns = []
for i in range(n_hours):
    columns = columns + past_cov_supervised.columns[i*past_cov.shape[1]:(i+1)*past_cov.shape[1]].tolist()
    columns = columns + past_ws_supervised.columns[i*stages.shape[1]:(i+1)*stages.shape[1]].tolist()
# columns

In [17]:
past_cov_supervised.reset_index(drop=True, inplace=True)
past_ws_supervised.reset_index(drop=True, inplace=True)

past_cov_ws_supervised = pd.concat([past_cov_supervised, past_ws_supervised], axis=1)
past_cov_ws_supervised = past_cov_ws_supervised[columns]


In [18]:
# past_cov_ws_supervised

In [19]:
shift_prior_known_supervised = prior_known_supervised.iloc[:, K*prior_known.shape[1]:]  # shift 2 days for prior known covariates
shift_prior_known_supervised.reset_index(drop=True, inplace=True)
shift_prior_known_supervised.columns = ['shift_prior_known_supervised_' + i for i in list(shift_prior_known_supervised.columns)]
shift_prior_known_supervised

,shift_prior_known_supervised_var1(t-48),shift_prior_known_supervised_var2(t-48),shift_prior_known_supervised_var3(t-48),shift_prior_known_supervised_var4(t-48),shift_prior_known_supervised_var5(t-48),shift_prior_known_supervised_var6(t-48),shift_prior_known_supervised_var7(t-48),shift_prior_known_supervised_var8(t-48),shift_prior_known_supervised_var9(t-48),shift_prior_known_supervised_var10(t-48),...,shift_prior_known_supervised_var16(t+23),shift_prior_known_supervised_var17(t+23),shift_prior_known_supervised_var18(t+23),shift_prior_known_supervised_var19(t+23),shift_prior_known_supervised_var20(t+23),shift_prior_known_supervised_var21(t+23),shift_prior_known_supervised_var22(t+23),shift_prior_known_supervised_var23(t+23),shift_prior_known_supervised_var24(t+23),shift_prior_known_supervised_var25(t+23)
0,7.00,1190.0,2902.0,24935.0,0.0,192.0,61.0,1038.0,1036.0,167.0,...,156.0,21.0,1182.0,132.0,2.0,1.0,0.0,7351.0,1.0,650.0
1,5.00,1023.0,2772.0,23214.0,0.0,189.0,60.0,1037.0,1036.0,167.0,...,163.0,12.0,1099.0,132.0,1.0,1.0,0.0,7403.0,1.0,938.0
2,4.00,1016.0,2936.0,22540.0,0.0,188.0,59.0,1037.0,1036.0,181.0,...,160.0,13.0,1076.0,108.0,2.0,2.0,0.0,7887.0,1.0,1021.0
3,4.00,1103.0,2893.0,22096.0,0.0,189.0,61.0,1037.0,1037.0,181.0,...,165.0,6.0,982.0,108.0,1.0,2.0,0.0,8348.0,1.0,2067.0
4,4.80,1120.0,2889.0,22066.0,0.0,190.0,59.0,1038.0,1037.0,171.0,...,162.0,20.0,982.0,108.0,1.0,2.0,0.0,8348.0,1.0,2067.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34963,66.00,2931.0,9149.0,32642.0,718.0,294.0,103.0,1028.0,1033.0,314.0,...,57.0,0.0,1135.0,30.0,1.0,2.0,0.0,3113.0,1.0,1.0
34964,62.09,2889.0,8024.0,32155.0,675.0,294.0,103.0,1028.0,1033.0,315.0,...,0.0,0.0,1172.0,30.0,3.0,1.0,1.0,3288.0,1.0,1.0
34965,61.77,2871.0,6871.0,30428.0,569.0,291.0,104.0,1029.0,1033.0,314.0,...,140.0,0.0,1148.0,50.0,4.0,3.0,1.0,3503.0,1.0,50.0
34966,58.43,2619.0,5822.0,28015.0,310.0,288.0,103.0,1029.0,1033.0,313.0,...,120.0,0.0,1128.0,60.0,5.0,2.0,1.0,3586.0,2.0,108.0


In [20]:
shift_prior_known_past_cov_ws_supervised = pd.concat([past_cov_ws_supervised, shift_prior_known_supervised], 
                                                     axis=1)


In [21]:
# shift_prior_known_past_cov_ws_supervised

In [22]:
future_ws_supervised = stages_supervised.iloc[:, n_hours*stages.shape[1]:]
future_ws_supervised.reset_index(drop=True, inplace=True)
future_ws_supervised

,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5),var1(t+6),var1(t+7),var1(t+8),var1(t+9),...,var1(t+14),var1(t+15),var1(t+16),var1(t+17),var1(t+18),var1(t+19),var1(t+20),var1(t+21),var1(t+22),var1(t+23)
0,50.54,48.68,48.02,47.06,46.79,47.63,47.44,50.84,54.84,59.95,...,52.07,52.15,61.12,71.98,74.64,75.38,76.49,73.83,69.51,70.77
1,48.68,48.02,47.06,46.79,47.63,47.44,50.84,54.84,59.95,59.35,...,52.15,61.12,71.98,74.64,75.38,76.49,73.83,69.51,70.77,64.89
2,48.02,47.06,46.79,47.63,47.44,50.84,54.84,59.95,59.35,57.03,...,61.12,71.98,74.64,75.38,76.49,73.83,69.51,70.77,64.89,60.91
3,47.06,46.79,47.63,47.44,50.84,54.84,59.95,59.35,57.03,59.22,...,71.98,74.64,75.38,76.49,73.83,69.51,70.77,64.89,60.91,59.68
4,46.79,47.63,47.44,50.84,54.84,59.95,59.35,57.03,59.22,58.56,...,74.64,75.38,76.49,73.83,69.51,70.77,64.89,60.91,59.68,58.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34963,73.35,73.64,71.92,69.49,68.40,60.32,57.73,57.66,57.04,57.48,...,72.12,71.81,72.42,71.80,71.95,70.85,71.36,75.13,77.61,77.02
34964,73.64,71.92,69.49,68.40,60.32,57.73,57.66,57.04,57.48,61.13,...,71.81,72.42,71.80,71.95,70.85,71.36,75.13,77.61,77.02,76.16
34965,71.92,69.49,68.40,60.32,57.73,57.66,57.04,57.48,61.13,60.70,...,72.42,71.80,71.95,70.85,71.36,75.13,77.61,77.02,76.16,74.30
34966,69.49,68.40,60.32,57.73,57.66,57.04,57.48,61.13,60.70,66.64,...,71.80,71.95,70.85,71.36,75.13,77.61,77.02,76.16,74.30,69.89


### Concatenation

In [23]:
all_data = concat([shift_prior_known_past_cov_ws_supervised, future_ws_supervised], axis=1)
# print("all_data", all_data)
print("all_data.shape:", all_data.shape)

all_data.shape: (34968, 3696)


### Train & Test set

In [24]:
all_data = all_data.values
n_train_hours = int(len(all_data)*0.8)
print("n_train_hours:", n_train_hours)


train = all_data[:n_train_hours, :]
test = all_data[n_train_hours:, :]

n_train_hours: 27974


In [25]:
n_obs = n_hours * (prior_known.shape[1]+data.shape[1])
n_obs

3672

### Normalization

In [26]:
# split into input and outputs
all_features = prior_known.shape[1] + data.shape[1]
n_obs = n_hours * all_features
train_X, train_y = train[:, :n_obs], train[:, n_obs:]
test_X, test_y = test[:, :n_obs], test[:, n_obs:]
print("train_X.shape, train_y.shape, test_X.shape, test_y.shape", train_X.shape, train_y.shape, test_X.shape, test_y.shape)


# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
train_X = scaler.fit_transform(train_X)
train_y = scaler.fit_transform(train_y)
test_X = scaler.fit_transform(test_X)
test_y = scaler.fit_transform(test_y)

train_X.shape, train_y.shape, test_X.shape, test_y.shape (27974, 3672) (27974, 24) (6994, 3672) (6994, 24)


In [27]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, all_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, all_features))
print("train_X.shape, train_y.shape, test_X.shape, test_y.shape: \n", train_X.shape, train_y.shape, test_X.shape, test_y.shape)

train_X.shape, train_y.shape, test_X.shape, test_y.shape: 
 (27974, 72, 51) (27974, 24) (6994, 72, 51) (6994, 24)


### Model

In [33]:
model_input = Input(shape=(train_X.shape[1], train_X.shape[2]))
x = layers.SimpleRNN(128, 
                     activation='relu', 
                     kernel_regularizer=keras.regularizers.L1L2(l1=1e-3, l2=1e-2),
                     recurrent_regularizer=keras.regularizers.L1L2(l1=1e-3, l2=1e-2),
                     return_sequences=True)(model_input)
x = layers.Flatten()(x)
# x = layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-4))(x)
# x = layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.L1L2(l1=1e-4, l2=1e-4))(x)
# x = layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-4))(x)

output = layers.Dense(train_y.shape[1])(x)

model_rnn = Model(model_input, output)
model_rnn.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 72, 51)]          0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 72, 128)           23040     
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 24)                221208    
Total params: 244,248
Trainable params: 244,248
Non-trainable params: 0
_________________________________________________________________


In [34]:
lr = 0.00001
EPOCHS = 8000

# initial_learning_rate * decay_rate ^ (step / decay_steps)
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-4, 
                                                          decay_steps=10000,
                                                          decay_rate=0.99)

model_rnn.compile(
              optimizer=Adam(learning_rate=lr_schedule), 
#               optimizer='adam',
              loss='mse',
              metrics=['mae'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=1000)
mc = ModelCheckpoint('saved_model/rnn_shift_anyday_24h_cp1.h5', monitor='val_mae', mode='min', verbose=2, save_best_only=True)


history = model_rnn.fit(train_X, train_y,
                    batch_size=512,
                    epochs=EPOCHS,
                    validation_data=(test_X, test_y),
                    verbose=2,
                    shuffle=True,
                       callbacks=[es, mc])

plt.rcParams["figure.figsize"] = (8, 6)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Epoch', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.legend(fontsize=14)
plt.title("Training loss vs Testing loss", fontsize=18)
# plt.savefig('graph/rnn_loss.png', dpi=300)
plt.show()

Epoch 1/8000

Epoch 00001: val_mae improved from inf to 0.23139, saving model to saved_model/rnn_shift_anyday_24h_cp1.h5
55/55 - 3s - loss: 3.7150 - mae: 0.1858 - val_loss: 3.6003 - val_mae: 0.2314
Epoch 2/8000

Epoch 00002: val_mae improved from 0.23139 to 0.19595, saving model to saved_model/rnn_shift_anyday_24h_cp1.h5
55/55 - 3s - loss: 3.4270 - mae: 0.1183 - val_loss: 3.3315 - val_mae: 0.1960
Epoch 3/8000

Epoch 00003: val_mae improved from 0.19595 to 0.17245, saving model to saved_model/rnn_shift_anyday_24h_cp1.h5
55/55 - 3s - loss: 3.1769 - mae: 0.1027 - val_loss: 3.0799 - val_mae: 0.1725
Epoch 4/8000

Epoch 00004: val_mae improved from 0.17245 to 0.15336, saving model to saved_model/rnn_shift_anyday_24h_cp1.h5
55/55 - 3s - loss: 2.9393 - mae: 0.0921 - val_loss: 2.8436 - val_mae: 0.1534
Epoch 5/8000

Epoch 00005: val_mae improved from 0.15336 to 0.13919, saving model to saved_model/rnn_shift_anyday_24h_cp1.h5
55/55 - 3s - loss: 2.7153 - mae: 0.0842 - val_loss: 2.6228 - val_mae: 0

Epoch 51/8000

Epoch 00051: val_mae did not improve from 0.07611
55/55 - 3s - loss: 0.0194 - mae: 0.0547 - val_loss: 0.0333 - val_mae: 0.1260
Epoch 52/8000

Epoch 00052: val_mae did not improve from 0.07611
55/55 - 3s - loss: 0.0178 - mae: 0.0543 - val_loss: 0.0331 - val_mae: 0.1308
Epoch 53/8000

Epoch 00053: val_mae did not improve from 0.07611
55/55 - 3s - loss: 0.0164 - mae: 0.0541 - val_loss: 0.0316 - val_mae: 0.1301
Epoch 54/8000

Epoch 00054: val_mae did not improve from 0.07611
55/55 - 3s - loss: 0.0151 - mae: 0.0538 - val_loss: 0.0281 - val_mae: 0.1215
Epoch 55/8000

Epoch 00055: val_mae did not improve from 0.07611
55/55 - 3s - loss: 0.0140 - mae: 0.0536 - val_loss: 0.0272 - val_mae: 0.1218
Epoch 56/8000

Epoch 00056: val_mae did not improve from 0.07611
55/55 - 2s - loss: 0.0131 - mae: 0.0533 - val_loss: 0.0272 - val_mae: 0.1251
Epoch 57/8000

Epoch 00057: val_mae did not improve from 0.07611
55/55 - 3s - loss: 0.0122 - mae: 0.0531 - val_loss: 0.0250 - val_mae: 0.1196
Epoch 

Epoch 109/8000

Epoch 00109: val_mae did not improve from 0.07611
55/55 - 3s - loss: 0.0061 - mae: 0.0479 - val_loss: 0.0132 - val_mae: 0.0908
Epoch 110/8000

Epoch 00110: val_mae did not improve from 0.07611
55/55 - 3s - loss: 0.0061 - mae: 0.0479 - val_loss: 0.0137 - val_mae: 0.0934
Epoch 111/8000

Epoch 00111: val_mae did not improve from 0.07611
55/55 - 3s - loss: 0.0061 - mae: 0.0480 - val_loss: 0.0147 - val_mae: 0.0977
Epoch 112/8000

Epoch 00112: val_mae did not improve from 0.07611
55/55 - 3s - loss: 0.0061 - mae: 0.0480 - val_loss: 0.0137 - val_mae: 0.0935
Epoch 113/8000

Epoch 00113: val_mae did not improve from 0.07611
55/55 - 3s - loss: 0.0061 - mae: 0.0479 - val_loss: 0.0125 - val_mae: 0.0876
Epoch 114/8000

Epoch 00114: val_mae did not improve from 0.07611
55/55 - 3s - loss: 0.0061 - mae: 0.0477 - val_loss: 0.0129 - val_mae: 0.0897
Epoch 115/8000

Epoch 00115: val_mae did not improve from 0.07611
55/55 - 3s - loss: 0.0061 - mae: 0.0476 - val_loss: 0.0123 - val_mae: 0.0867

Epoch 166/8000

Epoch 00166: val_mae did not improve from 0.07437
55/55 - 3s - loss: 0.0055 - mae: 0.0459 - val_loss: 0.0098 - val_mae: 0.0750
Epoch 167/8000

Epoch 00167: val_mae did not improve from 0.07437
55/55 - 3s - loss: 0.0055 - mae: 0.0458 - val_loss: 0.0104 - val_mae: 0.0785
Epoch 168/8000

Epoch 00168: val_mae improved from 0.07437 to 0.07414, saving model to saved_model/rnn_shift_anyday_24h_cp1.h5
55/55 - 3s - loss: 0.0055 - mae: 0.0458 - val_loss: 0.0096 - val_mae: 0.0741
Epoch 169/8000

Epoch 00169: val_mae did not improve from 0.07414
55/55 - 3s - loss: 0.0055 - mae: 0.0457 - val_loss: 0.0102 - val_mae: 0.0776
Epoch 170/8000

Epoch 00170: val_mae did not improve from 0.07414
55/55 - 3s - loss: 0.0054 - mae: 0.0457 - val_loss: 0.0099 - val_mae: 0.0755
Epoch 171/8000

Epoch 00171: val_mae improved from 0.07414 to 0.06764, saving model to saved_model/rnn_shift_anyday_24h_cp1.h5
55/55 - 3s - loss: 0.0055 - mae: 0.0459 - val_loss: 0.0086 - val_mae: 0.0676
Epoch 172/8000

Epoc

Epoch 222/8000

Epoch 00222: val_mae did not improve from 0.06498
55/55 - 2s - loss: 0.0051 - mae: 0.0447 - val_loss: 0.0085 - val_mae: 0.0683
Epoch 223/8000

Epoch 00223: val_mae did not improve from 0.06498
55/55 - 3s - loss: 0.0051 - mae: 0.0447 - val_loss: 0.0081 - val_mae: 0.0662
Epoch 224/8000

Epoch 00224: val_mae did not improve from 0.06498
55/55 - 3s - loss: 0.0051 - mae: 0.0447 - val_loss: 0.0085 - val_mae: 0.0683
Epoch 225/8000

Epoch 00225: val_mae did not improve from 0.06498
55/55 - 3s - loss: 0.0051 - mae: 0.0446 - val_loss: 0.0087 - val_mae: 0.0695
Epoch 226/8000

Epoch 00226: val_mae did not improve from 0.06498
55/55 - 3s - loss: 0.0051 - mae: 0.0446 - val_loss: 0.0082 - val_mae: 0.0668
Epoch 227/8000

Epoch 00227: val_mae improved from 0.06498 to 0.06409, saving model to saved_model/rnn_shift_anyday_24h_cp1.h5
55/55 - 3s - loss: 0.0052 - mae: 0.0449 - val_loss: 0.0078 - val_mae: 0.0641
Epoch 228/8000

Epoch 00228: val_mae improved from 0.06409 to 0.06291, saving mod

...


Epoch 05638: val_mae did not improve from 0.04311
55/55 - 3s - loss: 0.0032 - mae: 0.0373 - val_loss: 0.0042 - val_mae: 0.0450
Epoch 5639/8000

Epoch 05639: val_mae did not improve from 0.04311
55/55 - 3s - loss: 0.0032 - mae: 0.0373 - val_loss: 0.0046 - val_mae: 0.0492
Epoch 5640/8000

Epoch 05640: val_mae did not improve from 0.04311
55/55 - 3s - loss: 0.0032 - mae: 0.0376 - val_loss: 0.0044 - val_mae: 0.0475
Epoch 5641/8000

Epoch 05641: val_mae did not improve from 0.04311
55/55 - 3s - loss: 0.0032 - mae: 0.0373 - val_loss: 0.0043 - val_mae: 0.0460
Epoch 5642/8000

Epoch 05642: val_mae did not improve from 0.04311
55/55 - 3s - loss: 0.0032 - mae: 0.0373 - val_loss: 0.0045 - val_mae: 0.0479
Epoch 5643/8000

Epoch 05643: val_mae did not improve from 0.04311
55/55 - 3s - loss: 0.0032 - mae: 0.0374 - val_loss: 0.0045 - val_mae: 0.0482
Epoch 5644/8000

Epoch 05644: val_mae did not improve from 0.04311
55/55 - 3s - loss: 0.0032 - mae: 0.0373 - val_loss: 0.0041 - val_mae: 0.0444
Epoch 56

Epoch 5695/8000

Epoch 05695: val_mae did not improve from 0.04280
55/55 - 3s - loss: 0.0032 - mae: 0.0373 - val_loss: 0.0045 - val_mae: 0.0476
Epoch 5696/8000

Epoch 05696: val_mae did not improve from 0.04280
55/55 - 3s - loss: 0.0032 - mae: 0.0373 - val_loss: 0.0041 - val_mae: 0.0445
Epoch 5697/8000

Epoch 05697: val_mae did not improve from 0.04280
55/55 - 3s - loss: 0.0032 - mae: 0.0374 - val_loss: 0.0043 - val_mae: 0.0461
Epoch 5698/8000

Epoch 05698: val_mae did not improve from 0.04280
55/55 - 3s - loss: 0.0032 - mae: 0.0373 - val_loss: 0.0042 - val_mae: 0.0454
Epoch 5699/8000

Epoch 05699: val_mae did not improve from 0.04280
55/55 - 3s - loss: 0.0032 - mae: 0.0375 - val_loss: 0.0040 - val_mae: 0.0439
Epoch 5700/8000

Epoch 05700: val_mae did not improve from 0.04280
55/55 - 3s - loss: 0.0032 - mae: 0.0374 - val_loss: 0.0042 - val_mae: 0.0458
Epoch 5701/8000

Epoch 05701: val_mae did not improve from 0.04280
55/55 - 3s - loss: 0.0032 - mae: 0.0373 - val_loss: 0.0044 - val_mae:

Epoch 5752/8000

Epoch 05752: val_mae did not improve from 0.04280
55/55 - 3s - loss: 0.0032 - mae: 0.0373 - val_loss: 0.0043 - val_mae: 0.0463
Epoch 5753/8000

Epoch 05753: val_mae did not improve from 0.04280
55/55 - 3s - loss: 0.0032 - mae: 0.0372 - val_loss: 0.0043 - val_mae: 0.0466
Epoch 5754/8000

Epoch 05754: val_mae did not improve from 0.04280
55/55 - 3s - loss: 0.0032 - mae: 0.0374 - val_loss: 0.0042 - val_mae: 0.0458
Epoch 5755/8000

Epoch 05755: val_mae did not improve from 0.04280
55/55 - 3s - loss: 0.0032 - mae: 0.0373 - val_loss: 0.0041 - val_mae: 0.0443
Epoch 5756/8000

Epoch 05756: val_mae did not improve from 0.04280
55/55 - 3s - loss: 0.0032 - mae: 0.0373 - val_loss: 0.0042 - val_mae: 0.0458
Epoch 5757/8000

Epoch 05757: val_mae did not improve from 0.04280
55/55 - 3s - loss: 0.0032 - mae: 0.0373 - val_loss: 0.0042 - val_mae: 0.0451
Epoch 5758/8000

Epoch 05758: val_mae did not improve from 0.04280
55/55 - 3s - loss: 0.0032 - mae: 0.0372 - val_loss: 0.0041 - val_mae:

Epoch 5809/8000

Epoch 05809: val_mae did not improve from 0.04280
55/55 - 3s - loss: 0.0032 - mae: 0.0373 - val_loss: 0.0043 - val_mae: 0.0465
Epoch 5810/8000

Epoch 05810: val_mae did not improve from 0.04280
55/55 - 3s - loss: 0.0032 - mae: 0.0373 - val_loss: 0.0043 - val_mae: 0.0463
Epoch 5811/8000

Epoch 05811: val_mae did not improve from 0.04280
55/55 - 3s - loss: 0.0032 - mae: 0.0373 - val_loss: 0.0043 - val_mae: 0.0464
Epoch 5812/8000

Epoch 05812: val_mae did not improve from 0.04280
55/55 - 3s - loss: 0.0032 - mae: 0.0373 - val_loss: 0.0043 - val_mae: 0.0460
Epoch 5813/8000

Epoch 05813: val_mae did not improve from 0.04280
55/55 - 3s - loss: 0.0032 - mae: 0.0373 - val_loss: 0.0044 - val_mae: 0.0467
Epoch 5814/8000

Epoch 05814: val_mae did not improve from 0.04280
55/55 - 3s - loss: 0.0032 - mae: 0.0374 - val_loss: 0.0044 - val_mae: 0.0469
Epoch 5815/8000

Epoch 05815: val_mae did not improve from 0.04280
55/55 - 3s - loss: 0.0032 - mae: 0.0373 - val_loss: 0.0044 - val_mae:

KeyboardInterrupt: 

### Performance

In [35]:
from tensorflow.keras.models import load_model
saved_model = load_model('saved_model/rnn_shift_anyday_24h_cp1.h5')

In [36]:
yhat = saved_model.predict(test_X)
inv_yhat = scaler.inverse_transform(yhat)
inv_y = scaler.inverse_transform(test_y)

inv_yhat = pd.DataFrame(inv_yhat)
inv_y = pd.DataFrame(inv_y)
print("inv_y.shape, inv_yhat.shape", inv_y.shape, inv_yhat.shape)

error = inv_yhat - inv_y
print(error.shape)

inv_y.shape, inv_yhat.shape (6994, 24) (6994, 24)
(6994, 24)


In [37]:
print('MAE = {}'.format(float("{:.3f}".format(mae(inv_yhat.iloc[:, :], inv_y.iloc[:, :])))))
print('RMSE = {}'.format(float("{:.3f}".format(sqrt(mse(inv_yhat.iloc[:, :], inv_y.iloc[:, :]))))))

MAE = 3.478
RMSE = 4.686
